In [1]:
%run nbloader.py
import seaborn
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib import pylab as plt
import root_numpy
import pandas as pd
import math

from sklearn.svm import SVC
%load_ext Cython


BRICK_X = 124000
BRICK_Y = 99000
BRICK_Z = 75000
SAFE_M = 10000


In [2]:
from read_opera_bg import plot_bg;

importing notebook from read_opera_bg.ipynb
bg: 2767 tracks
len(slopes): 2767


In [3]:
from sklearn.svm import SVC
import pandas as pd

from read_opera_bg import load_bg;
from read_opera_mc import load_mc;
pbg = load_bg(step=1);
pmc = load_mc(step=1);

importing notebook from read_opera_mc.ipynb
numtracks reduction by cuts:  [188, 186, 109, 50, 19, 19]
bg: 27322110 tracks
numtracks reduction by cuts:  [18724, 18678, 11058, 5389, 2799, 2684]


In [4]:
def centralize_shower(shower):
    x_diff = shower['ele_x'] - 50000
    y_diff = shower['ele_y'] - 40000
    shower.loc['BT_X'] = shower.loc['BT_X'] - x_diff
    shower.loc['BT_Y'] = shower.loc['BT_Y'] - y_diff
    return shower


def generate_training_sample(pmc, pbg):
    z_coordinates = list(set(pbg['s.eZ']))
    z_coordinates.sort()
    def f(x): return x > 60000
    z_coordinates = filter(f, z_coordinates)
    
    data_to_clear = pbg[pbg['s.eZ'].isin(z_coordinates)]
    data_to_clear = data_to_clear.apply(np.random.permutation)
    data_to_clear['signal'] = 0
    data_to_clear = data_to_clear[:30000]
    
    
    parents = list()
    sons = list()

    for i in range(400):
        my_shower = pmc.iloc[i]
        shower_frame = pd.DataFrame([
            my_shower['BT_X'],
            my_shower['BT_Y'],
            my_shower['BT_Z'] - my_shower['BT_Z'] % 1293,
            my_shower['BT_SX'],
            my_shower['BT_SY']],
            index=['s.eX', 's.eY', 's.eZ', 's.eTX', 's.eTY']).T
        shower_frame['signal'] = 1
        shower_frame['s.eChi2'] = 0
        shower_frame = shower_frame[shower_frame['s.eZ'].isin(z_coordinates)]

        numpy_frame = np.asarray(shower_frame)

        for i in range(len(z_coordinates)):
            for track in numpy_frame[numpy_frame[:, 2] == z_coordinates[i] ,:]:
                next_layer = numpy_frame[numpy_frame[:, 2] == z_coordinates[i + 1] ,:]
                if (len(next_layer) != 0):
                    IP = compute_metric_distance(track, next_layer, classifier = False, IP_mode=True)
                    if (IP.min() < 0.001):
                        parents.append(track)
                        sons.append(next_layer[IP.argmin(), :])

    sons = np.asarray(sons)
    parents = np.asarray(parents)

    noise_sons = list()
    noise_parents = list()
    for i in range(10):
        number = np.random.randint(len(data_to_clear))
        track = data_to_clear.iloc[number, :]
        track = np.asarray(track)
        layer = data_to_clear[data_to_clear['s.eZ'] == track[2] + 1293]
        layer = np.asarray(layer)
        noise_sons.extend([track] * len(layer))

        noise_parents.append(layer)

    noise_sons = np.asarray(noise_sons)
    noise_parents = np.concatenate(noise_parents)
    
    signal_part = compute_features(sons, parents)
    noise_part = compute_features(noise_sons, noise_parents)
    
    signal_part = np.insert(signal_part, len(signal_part[0, :]),1,axis = 1)
    noise_part = np.insert(noise_part, len(noise_part[0, :]), 0,axis = 1)
    
    df = np.concatenate([signal_part, noise_part], axis = 0)
    return df

In [9]:
%%cython
import numpy as np
dZ = 205


def compute_features(sons, parents):
    distance = np.sqrt(np.sum((parents - sons)[:, 0:3]**2, axis = 1))
    tan_angle = np.tan(np.arccos(1/(np.sqrt(np.sum((sons - parents)[:, 3:5]**2, axis = 1) + 1))))

    angle_z_son = tan_angle
    if (len(sons.shape) == 1):
        track_begin = sons[0:3]
        track_end = np.copy(track_begin)
        track_end[0] += dZ * sons[3]
        track_end[1] += dZ * sons[4]
        track_end[2] += dZ
        track_diff = track_end - track_begin
        angle_z_son = (dZ ** 2)/(np.linalg.norm(track_diff) * dZ)
        angle_z_son = np.asarray([angle_z_son]*len(parents))
    else:
        track_begin = sons[:, 0:3]

        track_end = np.copy(track_begin)
        track_end[:, 0] += dZ * sons[:, 3]
        track_end[:, 1] += dZ * sons[:, 4]
        track_end[:, 2] += dZ
        track_diff = track_end - track_begin
        angle_z_son = (dZ ** 2)/(np.linalg.norm(track_diff, axis = 1) * dZ)
    
    
    track_diff_par = np.transpose(np.asarray([dZ * parents[:, 3], dZ * parents[:, 4]]))
    track_diff_par = np.insert(track_diff_par, 2, dZ, axis = 1)
    
    
    IP = np.linalg.norm(np.cross(parents[:, 0:3] - track_end, parents[:, 0:3] - track_begin), axis = 1)/np.linalg.norm(track_diff)

    angle_z_parents = (dZ ** 2)/(np.linalg.norm(track_diff_par, axis = 1) * dZ)
    return np.transpose(np.asarray([distance/dZ, tan_angle, IP/dZ, angle_z_son, angle_z_parents]))


def compute_metric_distance(track, next_layer, classifier, IP_mode = False):
    if (len(next_layer) == 0):
        return np.asarray([0])
    
    X = compute_features(track, next_layer)
    if IP_mode == True:
        return (X[:, 2]/30)**2

    return classifier.predict_proba(X)[:, 1]

def find_similar(track, next_layer, classifier, threshold, box_size):
    
    track = np.array(track, dtype = float)
    indexes = np.arange(len(next_layer))
    
    bool_index = (abs(next_layer[:, 0] - track[0]) < box_size) & (abs(next_layer[:, 1] - track[1]) < box_size)
    next_layer = next_layer[bool_index]
    indexes = indexes[bool_index]
    
    dist = compute_metric_distance(track, next_layer, classifier)
    result = dist.argmax()
    if dist[result] > threshold:
        return indexes[result]
    return -1



def clear_layer(to_clear, to_compare, classifier, threshold, box_size):
    if (len(to_clear) == 0):
        return to_clear
    result = np.apply_along_axis(find_similar, 1, to_clear, to_compare,  classifier, threshold, box_size)
    
    to_compare[result[result != -1], 6] = to_clear[result != -1, 6] + 1
    to_clear = to_clear[np.logical_or(result != -1, to_clear[:, 6] > 5)]
    numbers = np.unique(result[result != -1])
    to_compare = to_compare[numbers]

    
    return np.concatenate([to_clear, to_compare])

def shower_finder(data, classifier, number_of_iterations = 1, threshold = 0.98, box_size = 3000):
    layers = list(set(data[:, 2]))
    layers.sort()
    for n in range(number_of_iterations):
        for i in range(len(layers) - 1):
            cleared_data = clear_layer(data[data[:, 2] == layers[i]], data[data[:, 2] == layers[i+1]], classifier, threshold, box_size)
            data = data[(data[:, 2] != layers[i]) & (data[:, 2] != layers[i+1])]
            data = np.concatenate([data, cleared_data])
    return data


In [10]:
df = generate_training_sample(pmc, pbg)

In [11]:
svm = SVC(C = 0.3, kernel = 'poly', probability = True)

In [12]:
svm.fit(df[:, :5], df[:, 5])

SVC(C=0.3, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
#from sklearn.externals import joblib
#joblib.dump(svm, 'svm/svm_5_5.pkl');

In [14]:
z_coordinates = list(set(pbg['s.eZ']))
z_coordinates.sort()
def f(x): return x > 60000
z_coordinates = filter(f, z_coordinates)

data_to_clear = pbg[pbg['s.eZ'].isin(z_coordinates)]
data_to_clear = data_to_clear.apply(np.random.permutation)
data_to_clear['signal'] = 0
data_to_clear = data_to_clear[:500000]

In [16]:
%%time
for i in range(1):
    my_shower = pmc.iloc[15]
    my_shower = centralize_shower(my_shower.copy())
    shower_frame = pd.DataFrame([
                my_shower['BT_X'],
                my_shower['BT_Y'],
                my_shower['BT_Z'] - my_shower['BT_Z'] % 1293,
                my_shower['BT_SX'],
                my_shower['BT_SY']],
                index=['s.eX', 's.eY', 's.eZ', 's.eTX', 's.eTY']).T
    shower_frame['signal'] = 1
    shower_frame['s.eChi2'] = 0
    
    shower_frame = shower_frame[shower_frame['s.eZ'].isin(z_coordinates)]
    index=['s.eX', 's.eY', 's.eZ', 's.eTX', 's.eTY', 'signal']
    testing_data = pd.concat([shower_frame, data_to_clear])
    testing_data = testing_data[index]
    testing_data['number_of_ancestors'] = 0
    testing_data['parent_index'] = -1
    columns = testing_data.columns
    testing_data_1 = np.asarray(testing_data)
    cleared_data = shower_finder(testing_data_1, number_of_iterations = 3, classifier = svm, box_size = 1000, threshold = 0.975)
    cleared_data = pd.DataFrame(cleared_data,columns = columns)
    print "Original shower size: " + str(len(shower_frame))
    print "Cleared shower size: " + str(sum(cleared_data.signal == 1))
    print "Uncleared noise: " + str(sum(cleared_data.signal == 0))
    print ""

Original shower size: 314
Cleared shower size: 71
Uncleared noise: 14372

CPU times: user 1min 45s, sys: 1.4 s, total: 1min 47s
Wall time: 1min 48s


In [78]:
%matplotlib osx
plot_bg(cleared_data)

len(slopes): 14443


Кросс -- валидация не помогла, и я вернулся к изначальному сценарию. Более качественный датасет тоже сделал всё только хуже. Вероятно дело в том, что меня интересует достаточно специфичный сценарий (отсеить как можно больше шума, при этом пожертвав небольшим количеством сигнала), и обычные метрики качества здесь не подходят.

Из картинки видно, что классификатор улучшить сложно, есть множество последовательностей треков, которые выглядят так, как будто их оставила одна частица, и понять, что эта последовательность не относиться к сигналу, достаточно сложно. Более того, их станет ещё больше, когда мы увеличим плотность шума.

Я предлагаю попробовать следующую вещь:
1. Первоночально пройтись по кирпичу существующим алгроритмом и получить набор последовательностей.
2. Пройтись по кирпичу и оставить только последовательность длиннее 5, например (уже должно быть не плохо)
3. Далее, если этого будет недостаточно, чтобы увидеть сигнал, мы построим классификатор, который должен будет классифицировать не треки, как раньше, а последовательность целиком. Фичи могут быть: количество треков, расстояние до ближайшей последовательности и тд.

In [18]:
def produce_sequences(cleared_data, z_coordinates, classifier):
    result_data = cleared_data[cleared_data[:,2] == z_coordinates[0]]
    result_data = result_data.reshape((len(result_data),1,len(result_data[0, :])))
    
    for i in range(len(z_coordinates) - 1):
        next_layer = cleared_data[cleared_data[:,2] == z_coordinates[i + 1]]
        additional = -np.ones((len(result_data),1,len(result_data[0,0, :])))
        
        result_data = np.concatenate([result_data, additional], axis = 1)
        result = np.apply_along_axis(find_similar, 1, result_data[:,i,:], next_layer,  classifier, 0.95, 1000)
        result_data[result != -1, i + 1, :] = next_layer[result[result != -1]]
    
    return result_data

In [19]:
sequences = produce_sequences(np.asarray(cleared_data), z_coordinates, classifier = svm)

In [20]:
length_array = np.zeros(len(sequences))

In [21]:
length_array = length_array.astype(int)

In [22]:
for i in range(len(sequences)):
    length_array[i] = len(sequences[i, :, 0])
    for n in range(len(sequences[i, :, 0])):
        if sequences[i, n, 0] == -1:
            length_array[i] = n
            break


In [23]:
means_coordinates = np.zeros((len(sequences), 2))

In [24]:
for i in range(len(sequences)):
    means_coordinates[i, 0] = np.mean(sequences[i,:length_array[0],0])
    means_coordinates[i, 1] = np.mean(sequences[i,:length_array[0],1])

In [25]:
colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
colors = np.hstack([colors] * 20)

In [26]:
df = np.concatenate([means_coordinates, length_array.reshape((len(length_array), 1))], axis = 1)

In [89]:
df_subset = df[df[:, 2] > 4]

In [90]:
len(df_subset)

1186

In [91]:
from sklearn.cluster import DBSCAN

In [92]:
clustr = DBSCAN(eps=900)

In [93]:
clustr.fit(df_subset[:, :2])

DBSCAN(algorithm='auto', eps=900, leaf_size=30, metric='euclidean',
    min_samples=5, p=None, random_state=None)

In [94]:
y_pred = clustr.fit_predict(df_subset[:, :2])

In [95]:
len(set(y_pred))

72

In [97]:
plt.scatter(df_subset[:, 0], df_subset[:, 1], color=colors[y_pred].tolist());

In [85]:
small_seq = sequences[length_array > 5]

In [86]:
small_seq.shape

(1186, 11, 8)

In [87]:
small_seq = small_seq.reshape((small_seq.shape[0] * 11, 8))

In [88]:
small_seq = small_seq[small_seq[:, 0] != -1]

In [68]:
small_seq_pd = pd.DataFrame(small_seq,columns = columns)

In [84]:
plot_bg(small_seq_pd)

len(slopes): 3657


 Итак, после первичной обработки данных у нас остается слишком много шума. Более того, сгустки сигнала практически неотличимы от душа. Что мы имеем? 500 объектов, похожих друг на друга. Нас нужно найти небольшое подмножество (~10 объектов), которое отличается лишь тем, что имеет некоторую геометрическую структуру относительно остальных подобных подгрупп.
 Идея: можно с помощью DBSCAN получить некоторое количество групп. Скажем 20. И после этого пытаться классифицировать их. Какие могут быть фичи: количество объектов, средняя длина, какие нибудь углы. 